| Submitted By    | Amity Lu        |
| --------------- | --------------- |
| Github Username | Amitywen        |


In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.metrics import hamming_loss
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_score
from sklearn.multiclass import OneVsRestClassifier

In [2]:
df = pd.read_csv('Frogs_MFCCs.csv')
df

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7190,1.0,-0.554504,-0.337717,0.035533,0.034511,0.443451,0.093889,-0.100753,0.037087,0.081075,...,0.069430,0.071001,0.021591,0.052449,-0.021860,-0.079860,Hylidae,Scinax,ScinaxRuber,60
7191,1.0,-0.517273,-0.370574,0.030673,0.068097,0.402890,0.096628,-0.116460,0.063727,0.089034,...,0.061127,0.068978,0.017745,0.046461,-0.015418,-0.101892,Hylidae,Scinax,ScinaxRuber,60
7192,1.0,-0.582557,-0.343237,0.029468,0.064179,0.385596,0.114905,-0.103317,0.070370,0.081317,...,0.082474,0.077771,-0.009688,0.027834,-0.000531,-0.080425,Hylidae,Scinax,ScinaxRuber,60
7193,1.0,-0.519497,-0.307553,-0.004922,0.072865,0.377131,0.086866,-0.115799,0.056979,0.089316,...,0.051796,0.069073,0.017963,0.041803,-0.027911,-0.096895,Hylidae,Scinax,ScinaxRuber,60


In [3]:
# check the data value
df.isnull().values.any()
Label = ['Family', 'Genus', 'Species']
Label

['Family', 'Genus', 'Species']

In [4]:
def splite_the_data(data, label, index):
    X = data.drop(columns = label[index])
    Y = data[label[index]]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)
    return X_train, X_test, y_train, y_test
    

In [5]:
# reference: https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
# reference: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.hamming_loss.html

def hamLoss(estimator, X_test, y_test):
    #StandardScaler()
    clf = make_pipeline(estimator)
    clf.fit(X_test, y_test)
    y_pred = clf.predict(X_test)
    return 1-(hamming_loss(y_test, y_pred))

In [6]:
def hamLossNOr(estimator, X_test, y_test):
    clf = make_pipeline(StandardScaler(),estimator)
    clf.fit(X_test, y_test)
    y_pred = clf.predict(X_test)
    return 1-(hamming_loss(y_test, y_pred))

In [7]:
family = df.drop(columns=['Genus', 'Species'])
fa_trainX, fa_testX, fa_trainY, fa_testY = splite_the_data(family,Label,0)

Genus = df.drop(columns=['Family', 'Species'])
Ge_trainX, Ge_testX, Ge_trainY, Ge_testY = splite_the_data(Genus,Label,1)

Species = df.drop(columns=['Genus', 'Family'])
Sp_trainX, Sp_testX, Sp_trainY, Sp_testY = splite_the_data(Species,Label,2)

In [8]:
# # reference: https://scikit-learn.org/stable/modules/cross_validation.html
# # reference: https://www.analyticsvidhya.com/blog/2021/04/insight-into-svm-support-vector-machine-along-with-code/#:~:text=We%20need%20to%20understand%20its,correct%20value%20using%20Hyperparameter%20Tuning.
# # reference: https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html#sphx-glr-auto-examples-svm-plot-rbf-parameters-py
# def DecideC(X, y, fold_num):
#     c = [0.001, 0.01, 0.1, 1, 10, 100]
#     for i in range(6):
#         clf = svm.SVC(decision_function_shape='ovr', kernel='rbf', C = c[i])
#         scores = cross_val_score(clf, X, y, cv=fold_num)
#         print("C = %0.2f , %0.2f accuracy with a standard deviation of %0.2f" % (c[i], scores.mean(), scores.std()))

trainX_set = [fa_trainX,Ge_trainX,Sp_trainX]
trainY_set = [fa_trainY,Ge_trainY,Sp_trainY]

testX_set = [fa_testX,Ge_testX,Sp_testX,]
testY_set = [fa_testY,Ge_testY,Sp_testY]

# for i in range(3):
#     print(Label[i])
#     DecideC(trainX_set[i],trainY_set[i],10)

In [9]:
def DecideCandW(X, y):
    C_range = np.logspace(-3,3,7)
    gamma_range = np.logspace(-3,3,7)
    param_grid = dict(gamma=gamma_range, C=C_range)
    grid = GridSearchCV(SVC(), param_grid=param_grid, cv=10, scoring=hamLoss) #SVC defalut rbf, decision_function_shape='ovr'
    grid.fit(X, y)

    print(
    "The best parameters are %s with a score of %0.2f"
    % (grid.best_params_, grid.best_score_)
)

In [10]:
for i in range(3):
    print(Label[i])
    DecideCandW(trainX_set[i],trainY_set[i])

Family
The best parameters are {'C': 1.0, 'gamma': 10.0} with a score of 1.00
Genus
The best parameters are {'C': 1.0, 'gamma': 1.0} with a score of 1.00
Species
The best parameters are {'C': 1.0, 'gamma': 1.0} with a score of 1.00


In [11]:
# reference: https://scikit-learn.org/stable/modules/generated/sklearn.svm.l1_min_c.html#sklearn.svm.l1_min_c
# reference: https://scikit-learn.org/stable/modules/kernel_approximation.html#kernel-approximation
# reference: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV

def DecideCandWwithL1(X, y):
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)
    gamma_range = np.logspace(-3,3,7)
    param_grid = dict(gamma=gamma_range)
    svc = svm.SVC(C = svm.l1_min_c(X, y,loss='squared_hinge', fit_intercept=True, intercept_scaling=1.0), kernel='rbf', decision_function_shape='ovr')
    grid = GridSearchCV(svc, param_grid=param_grid, cv=10, scoring=hamLossNOr) #SVC defalut rbf, decision_function_shape='ovr'
    grid.fit(X, y)

    print(
    "The best parameters are %s with a score of %0.2f"
    % (grid.best_params_, grid.best_score_)
)

def L1SMV(X, y):
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)
    clf = svm.SVC(C = svm.l1_min_c(X, y,loss='squared_hinge', fit_intercept=True, intercept_scaling=1.0), kernel='rbf', decision_function_shape='ovr')
    scores = cross_val_score(clf, X, y, cv=10)
    print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

In [12]:
print('Finding the best gamma and C')
for i in range(3):
    print(Label[i])
    DecideCandWwithL1(trainX_set[i],trainY_set[i])

print('using the auto definition')
for i in range(3):
    print(Label[i])
    L1SMV(trainX_set[i],trainY_set[i])

Finding the best gamma and C
Family
The best parameters are {'gamma': 0.001} with a score of 0.62
Genus
The best parameters are {'gamma': 0.001} with a score of 0.58
Species
The best parameters are {'gamma': 0.001} with a score of 0.49
using the auto definition
Family
0.62 accuracy with a standard deviation of 0.00
Genus
0.58 accuracy with a standard deviation of 0.00
Species
0.49 accuracy with a standard deviation of 0.00


In [13]:
# reference: https://stackoverflow.com/questions/73785500/what-does-random-state-do-in-regards-to-smote-bsmote-adasyn-python-libra

sm = SMOTE(random_state=42)

print('Finding the best gamma and C')
for i in range(3):
    print('resampling', Label[i],'....')
    X_res, y_res = sm.fit_resample(trainX_set[i],trainY_set[i])
    DecideCandWwithL1(X_res, y_res)


print('using the auto definition')
for i in range(3):
    print('resampling', Label[i],'....')
    X_res, y_res = sm.fit_resample(trainX_set[i],trainY_set[i])
    L1SMV(X_res, y_res)

Finding the best gamma and C
resampling Family ....
The best parameters are {'gamma': 0.1} with a score of 0.30
resampling Genus ....
The best parameters are {'gamma': 0.1} with a score of 0.19
resampling Species ....


KeyboardInterrupt: 

**Conclusion:**
> The classfier in 6(b)ii, without penalty, this model is overfitting.    
> The classfier in 6(b)iii, with penalty, it looks better.   
> When we use SMOTE to balanced the data, we can find out that only the label family has better result. Other labels don't get better hamming loss. 